#Convolution and its effects in the time/frequency domain

#Introduction
-Run this code. A self-explanatory graphical user interface should pop up.

In [ ]:
import sys
import Tkinter as tk
import matplotlib
import ttk as ttk
#matplotlib.use("TkAgg")

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg

import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from math import floor
from matplotlib import style
style.use("ggplot")


LARGE_FONT = ("Verdana", 12)
"""
The code is divided in three big parts. The first part are all the functions that do the calculations and plotting on the
first page. The second part are the functions for the second page. The third part is the application itself
(class Application) and the defining of the pages etc.
"""


#FIRST PART: FUNCTIONS USED IN THE FIRST PAGE
def ConVol(self,ff,sf):
    """
    Function to plot the initial graphs and the slider on the first page when the functions are chosen and the OK button
    is pressed.
    ff and sf are the numbers of respectively the first and the second function
    """

    global xsf, xff, ysf, yff, fnumber1, fnumber2
    fnumber1 = ff
    fnumber2 = sf

    #ff = f1.get() First function

    if ff == 1:  #Rectangle
         xff = np.linspace(0,1,1000)
         yff = [0,] + [1,]*998 + [0,]

    elif ff == 2: #Triangle
        xff = np.linspace(0,1,1000)
        yff = list(np.linspace(0,1,500)) + list(np.linspace(1,0,500))

    elif ff == 3: #Sinus
        xff = np.linspace(-3,3,6000)
        yff = 0.5*np.sin(xff*np.pi)+0.5

    elif ff == 4: #Dirac impulse train  (3 impulses)
         xff = [-1.2,]*100 + list(np.linspace(-1.3,-1.2,10)) + list(np.linspace(-1.2,-1.1,10))
         yff = list(np.linspace(0,1,100))+list(np.linspace(0.9,1,10)) + list(np.linspace(1,0.9,10))

            #sf = f2.get() #second function

    if sf == 1:  #Rectangle
         xsf = np.linspace(0,1,1000)
         ysf = [0,] + [1,]*998 + [0,]

    elif sf == 2: #Triangle
         xsf = np.linspace(0,1,1000)
         ysf = list(np.linspace(0,1,500)) + list(np.linspace(1,0,500))

    elif sf == 3: #Sinus
         xsf = np.linspace(-3,3,6000)
         ysf = 0.5*np.sin(xsf*np.pi)+0.5


    fig1 = plt.figure(figsize=(10,6))
    global axes1
    axes1 = fig1.add_subplot(111)
    axes1.clear()
    if ff<4:
        axes1.plot(xff,yff,'r',label = 'f(tau)')
    elif ff == 4 :
        axes1.plot(xff,yff,'r',label = 'f(tau)')
        axes1.plot(np.asarray(xff)+1.2,yff,'r')
        axes1.plot(np.asarray(xff)+2.4,yff,'r')


    #Defining a global variable that can later be used to update the graphs (see function Slide)
    global my_line
    my_line, = axes1.plot(-xsf-1.5,ysf,'b',label = 'g(t-tau)')

    axes1.set_xlim([-2.5,3])
    axes1.set_ylim([0,1.5])

    #The initial convolution value (for t = -1.5, the leftmost value of the slider)
    if (ff < 3 and sf < 3) or (ff==4 and sf < 3):
        y = 0
    elif (ff == 3 and sf == 1) or (ff == 1 and sf == 3):
        y = 0.49974
    elif (ff == 3 and sf ==2) or (ff == 2 and sf == 3):
        y = 0.25046638
    elif (ff == 3 and sf == 3):
        y = 1.006772
    elif ff == 4 and sf == 3:
        y = 1.190983

    axes1.plot(-1.5,y,'go',label = 'h(t) = (f*g)(t)')

    axes1.legend()
    plt.xlabel("tau (s)")
    global canvas1
    canvas1 = FigureCanvasTkAgg(fig1,self)
    canvas1.show()
    canvas1.get_tk_widget().place(x = 200, y = 110)


    #Creating the slider
    label1 = tk.Label(self,text = "t",font=('Verdana',10)).place(x=290,y = 80)
    tscale = tk.Scale(self,from_=-1.5, to=3,orient=tk.HORIZONTAL, resolution = 0.01,length = 500,command = Slide)
    tscale.place(x=300,y=60)
    tscale.set(-1.5)


def PlotConVol(self,ff,sf):
    """
    This function is called when the 'Plot the resulting convolution' button is pressed. It plots the interpolation of
    the convolution points drawn by the slider as a dashed line.
    """


    global axes1, canvas1, xff, xsf, yff, ysf
    if ff < 3 and sf < 3:  #rectangle and triangle

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        xcon = np.linspace(0,2,len(xff)+len(xsf)-1)

    elif ff == 4 and sf == 1:        #Impulse train and rectangle

        xcon = 0.5 + np.asarray([-2.5,-1.7,-1.7,-0.7,-0.7,-0.5,-0.5,0.5,0.5,0.7,0.7,1.7,1.7,2.5])
        ycon = [0,0,1,1,0,0,1,1,0,0,1,1,0,0]

    elif ff == 4 and sf == 2:         #Impulse train and triangle

        xcon = 0.5+np.asarray([-2.5,-1.7,-1.2,-0.7,-0.5,0,0.5,0.7,1.2,1.7,2.5])
        ycon = [0,0,1,0,0,1,0,0,1,0,0]

    elif ff == 4 and sf == 3:           #Impulse train and sinus

        xcon = np.linspace(-2.5,1.8,1000)
        ycon = 0.5*np.sin(np.pi*(xcon+1.2))+0.5*np.sin(np.pi*xcon) + 0.5*np.sin(np.pi*(xcon-1.2)) + 1.5
        xcon2 = np.linspace(1.8,3,1000)
        ycon2 = 0.5*np.sin(np.pi*xcon2) + 0.5*np.sin(np.pi*(xcon2-1.2)) + 1.0
        xcon = list(xcon)+list(xcon2)
        ycon = list(ycon)+ list(ycon2)


    elif (ff == 3 and sf < 3) or (ff<3 and sf ==3) :            #Sinus and rectangle/triangle

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        xcon = np.linspace(-3,4,len(ycon))

        ycon[0:1000] = ycon[2000:3000]


    elif ff == 3 and sf == 3:         #Sinus and sinus

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        xcon = np.linspace(-6,6,len(ycon))

    axes1.plot(xcon,ycon,'g--')
    canvas1.draw()

def Slide(val):
    """
    This function is called when you slide the slider on the first page. It updates g(t-tau) and calculates the value of
    the convolution at that point.
    """


    t = eval(val)

    global xsf, xff, ysf, yff
    global axes1
    global fnumber1, fnumber2
    global canvas1
    global my_line
    my_line.set_xdata(-xsf+t)

    if fnumber1 < 3 and fnumber2 < 3:  #rectangle and triangle

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        #xcon = np.linspace(-0.5,1.5,len(xff)+len(xsf)-1)

        yconex = [0,]*1501+list(ycon)+[0,]*1001

        yconext = yconex[int((t+1.5)*1000)]
        axes1.plot(t,yconext,'go')

    elif fnumber1 == 4 and fnumber2 == 1:        #Impulse train and rectangle
        if t in [-1.2,-0.2,0,1,1.2,2.2]:
            axes1.plot([t,]*100,np.linspace(0,1,100),'go')
        elif (t>-1.2 and t<-0.2) or (t>0 and t <1) or (t>1.2 and t<2.2):
            axes1.plot(t,1,'go')
        else:
            axes1.plot(t,0,'go')

    elif fnumber1 == 4 and fnumber2 == 2:         #Impulse train and triangle
        if (t>-1.2 and t<=-0.7):
            axes1.plot(t,t/0.5+2.4,'go')
        elif (t>-0.7 and t<-0.2):
            axes1.plot(t,-t/0.5-0.4,'go')
        elif (t>0 and t<=0.5):
            axes1.plot(t,t/0.5,'go')
        elif (t>0.5 and t<1):
            axes1.plot(t,-t/0.5+2,'go')
        elif (t>1.2 and t<=1.7):
            axes1.plot(t,t/0.5-2.4,'go')
        elif (t>1.7 and t<2.2):
            axes1.plot(t,-t/0.5+4.4,'go')
        else:
            axes1.plot(t,0,'go')

    elif fnumber1 == 4 and fnumber2 == 3:           #Impulse train and sinus
        if t <= 1.8:
            y = 0.5*np.sin(np.pi*(t+1.2))+0.5*np.sin(np.pi*t) + 0.5*np.sin(np.pi*(t-1.2)) + 1.5
        else:
            y = 0.5*np.sin(np.pi*t) + 0.5*np.sin(np.pi*(t-1.2)) + 1
        axes1.plot(t,y,'go')
        axes1.set_ylim([0,2])

    elif (fnumber1 == 3 and fnumber2 < 3) or (fnumber1<3 and fnumber2 ==3) :            #Sinus and rectangle/triangle

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        ycon = list(ycon)
        ycon = ycon[1500:6001]
        ycont = ycon[int((t+1.5)*1000)]
        axes1.plot(t,ycont,'go')

    elif fnumber1 == 3 and fnumber2 == 3:         #Sinus and sinus

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        ycon = list(ycon)
        ycon = ycon[4500:9001]
        ycont = ycon[int((t+1.5)*1000)]
        axes1.plot(t,ycont,'go')
        axes1.set_ylim([0,2.5])


    canvas1.draw()





#SECOND PART: FUNCTIONS FOR THE SECOND PAGE

def PrintTime(self):
    """
    Prints the time labels above the function choices if convolution in the time domain is chosen.
    """


    label2 = ttk.Label(self, text = "               ").place(x = 40,y = 220)
    label3 = ttk.Label(self, text = "               ").place(x = 130,y = 220)
    label2 = ttk.Label(self, text = "f(t)",font=('Verdana',10)).place(x = 40,y = 220)
    label3 = ttk.Label(self, text = "g(t)",font=('Verdana',10)).place(x = 130,y = 220)

def PrintFreq(self):
    """
    Prints the frequency labels above the function choices if convolution in the frequency domain is chosen.
    """

    label2 = ttk.Label(self, text = "               ").place(x = 40,y = 220)
    label3 = ttk.Label(self, text = "               ").place(x = 130,y = 220)
    label2 = ttk.Label(self, text = "F(jw)",font=('Verdana',10)).place(x = 40,y = 220)
    label3 = ttk.Label(self, text = "G(jw)",font=('Verdana',10)).place(x = 130,y = 220)

def ConVolTF(self, ff, sf, TF):
    """
    The main function that plots all the graphs after the functions are chosen and the OK button is pressed.
    ff is the number of the first function
    sf is the number of the second function
    TF indicates whether the variable of the chosen functions is the time t (TF == 1) or the frequency w (TF == 2)
    """


    global xsf, xff, ysf, yff
    label1 = ttk.Label(self,text = "                                                                    ",font = ('Verdana',10)).place(x = 500,y = 60)
    label2 = ttk.Label(self,text = "                                                                    ",font = ('Verdana',10)).place(x=500, y=350)

    if TF == 1:
        label1 = ttk.Label(self,text = "Time domain: Convolution",font = ('Verdana',10)).place(x = 500,y = 60)
        label2 = ttk.Label(self,text = "Frequency domain: Multiplication",font = ('Verdana',10)).place(x=500, y=350)
    elif TF == 2:
        label1 = ttk.Label(self,text = "Time domain: Multiplication (*2pi)",font = ('Verdana',10)).place(x = 500,y = 60)
        label2 = ttk.Label(self,text = "Frequency domain: Convolution",font = ('Verdana',10)).place(x=500, y=350)

    label3 = ttk.Label(self,text = "=", font = ("Verdana",24)).place(x = 825,y=160)
    label4 = ttk.Label(self,text = "=", font = ("Verdana",24)).place(x = 825,y=450)

    if ff == 1:  #Rectangle
         xff = np.linspace(-0.5,0.5,1000)
         yff = [0,] + [1,]*998 + [0,]

    elif ff == 2: #Triangle
        xff = np.linspace(-0.5,0.5,1000)
        yff = list(np.linspace(0,1,500)) + list(np.linspace(1,0,500))

    elif ff == 3: #Sinus
        xff = np.linspace(-5,5,10000)
        yff = np.sin(xff*np.pi)

    elif ff == 4: #Dirac impulse train
         xff = [-1.2,]*100 + list(np.linspace(-1.3,-1.2,10)) + list(np.linspace(-1.2,-1.1,10))
         yff = list(np.linspace(0,1,100))+list(np.linspace(0.9,1,10)) + list(np.linspace(1,0.9,10))

    elif ff == 5: #Sinc
        xff = np.linspace(-6*np.pi,6*np.pi,10000)
        yff = 2*np.sin(xff/2)/xff


    if sf == 1:  #Rectangle
         xsf = np.linspace(-0.5,0.5,1000)
         ysf = [0,] + [1,]*998 + [0,]

    elif sf == 2: #Triangle
         xsf = np.linspace(-0.5,0.5,1000)
         ysf = list(np.linspace(0,1,500)) + list(np.linspace(1,0,500))

    elif sf == 3: #Sinus
         xsf = np.linspace(-5,5,10000)
         ysf = np.sin(xsf*np.pi)

    elif sf == 5: #Sinc
         xsf = np.linspace(-6*np.pi,6*np.pi,10000)
         ysf = 2*np.sin(xsf/2)/xsf


    #Creating some labels and legends for the graphs
    if TF == 1:
        y1,y2 = 80,370
        textf1,textf2 = 'f(t)','F(jw)'
        textg1,textg2 = 'g(t)','G(jw)'
        texth1,texth2 = 'h(t) = (f*g)(t)','H(jw)'
        Sign1, Sign2 = "*","x"
        size1, size2 = 24,20
    elif TF == 2:
        y1,y2 = 370,80
        textf1,textf2 = 'F(jw)','f(t)'
        textg1,textg2 = 'G(jw)','g(t)'
        texth1,texth2 = 'H(jw) = (F*G)(jw)','h(t)'
        Sign1, Sign2 = "x","*"
        size1, size2 = 20, 24
    ttk.Label(self,text = "    ").place(x = 490,y = 160)
    ttk.Label(self,text = Sign1, font = ("Verdana",size1)).place(x = 495, y = 160)
    ttk.Label(self,text = "    ").place(x = 490,y = 450)
    ttk.Label(self,text = Sign2, font = ("Verdana",size2)).place(x = 495, y = 450)


    #Axes1 is the plot of f(t) (or F(jw) if frequency domain is chosen)
    fig1 = plt.figure(figsize=(3.5,2.5))
    global axes1
    axes1 = fig1.add_subplot(111)
    axes1.clear()

    if ff<4 or ff == 5:
        axes1.plot(xff,yff,'r',label = textf1)
    elif ff == 4 :
        axes1.plot(xff,yff,'r',label = textf1)
        axes1.plot(np.asarray(xff)+1.2,yff,'r')
        axes1.plot(np.asarray(xff)+2.4,yff,'r')
        axes1.plot(np.asarray(xff)+3.6,yff,'r')
        axes1.plot(np.asarray(xff)-1.2,yff,'r')
        axes1.text(0,1,'$1$')

    ylimmin = 0
    if ff == 3:
        ylimmin = -1.5
    elif ff == 5:
        ylimmin = -0.5
    xlim = 2.5
    if ff == 5:
        xlim = 6*np.pi

    axes1.set_xlim([-xlim,xlim])
    axes1.set_ylim([ylimmin,1.5])
    draw_axes(axes1,-xlim,xlim,ylimmin,1.5)
    axes1.legend()


    global canvas1
    canvas1 = FigureCanvasTkAgg(fig1,self)
    canvas1.show()
    canvas1.get_tk_widget().place(x = 200, y = y1)


    #Axes2 is the plot of g(t) (respectively G(jw) if freq dom is chosen)
    fig2 = plt.figure(figsize=(3.5,2.5))
    global axes2
    axes2 = fig2.add_subplot(111)
    axes2.clear()
    ylimmin = 0
    if sf == 3:
        ylimmin = -1.5
    elif sf == 5:
        ylimmin = -0.5
    xlim = 2.5
    if sf == 5:
        xlim = 6*np.pi

    axes2.plot(xsf,ysf,'b',label = textg1)
    axes2.set_xlim([-xlim,xlim])
    axes2.set_ylim([ylimmin,1.5])
    draw_axes(axes2,-xlim,xlim,ylimmin,1.5)
    axes2.legend()

    global canvas2
    canvas2 = FigureCanvasTkAgg(fig2,self)
    canvas2.show()
    canvas2.get_tk_widget().place(x = 520, y = y1)



    #Axes3 is plot of the convolution using the function CalcConvolution
    fig3 = plt.figure(figsize=(3.5,2.5))
    global axes3
    axes3 = fig3.add_subplot(111)
    axes3.clear()
    x3,y3 = CalcConvolution(ff,sf)
    axes3.plot(x3,y3,'g',label=texth1)

    ylim1 = 0
    ylim2 = 1.5
    if ff == 3 or sf == 3:
        ylim1 = -1
    if ff==3 and sf ==3:
        ylim1 = -2
        ylim2 = 2
    if ff==4 and sf==3:
        ylim1 = -1
        ylim2 = 1
    if ff == 5 or sf == 5:
        ylim1 = -0.5
        ylim2 = 1.5
    if ff == 5 and sf == 5:
        ylim1 = -1.5
        ylim2 = 9
    if ff == 4 and sf == 5:
        ylim1 = 0
        ylim2 = 8

    xlim1 = -2.5
    xlim2 = 2.5
    if ff == 5 or sf == 5:
        xlim1 = -6*np.pi
        xlim2 = 6*np.pi

    axes3.set_xlim([xlim1, xlim2])
    axes3.set_ylim([ylim1,ylim2])
    draw_axes(axes3,xlim1,xlim2,ylim1,ylim2)
    axes3.legend()


    global canvas3
    canvas3 = FigureCanvasTkAgg(fig3,self)
    canvas3.show()
    canvas3.get_tk_widget().place(x = 870, y = y1)



    #Axes4 is the plot of the (inverse) Fourier transform of f(t) (F(jw)) calculated by the function FourierTransform
    fig4 = plt.figure(figsize=(3.5,2.5))
    global axes4
    axes4 = fig4.add_subplot(111)
    axes4.clear()

    Xff,Yff,Xlim1,Ylim1 = FourierTransform(ff,TF)
    if ff == 3:
        Xff2 = Xff[1]
        Yff2 = Yff[1]
        axes4.plot(Xff2,Yff2,'r')
        Xff = Xff[0]
        Yff = Yff[0]
        if TF == 1:
            axes4.text(-3,2.5,'$(j\pi)$',fontsize = 8)
            axes4.text(2,-3,'$(-j\pi)$',fontsize = 8)
        elif TF ==2:
            axes4.text(-3,-0.5,"$(-j1/2)$")
            axes4.text(2,0.4,"$(j1/2)$")
    elif ff == 4:
        Xff3 = np.asarray(Xff) +2*2*np.pi/1.2
        Yff3 = Yff
        axes4.plot(Xff3,Yff3,'r')
        Xff2 = np.asarray(Xff) + 2*np.pi/1.2
        Yff2 = Yff
        axes4.plot(Xff2,Yff2,'r')
        if TF == 1:
            axes4.text(0,2*np.pi/1.2,"$2\pi/1.2$")
        elif TF == 2:
            axes4.text(0,1/1.2,"$1/1.2$")
    axes4.plot(Xff,Yff,'r',label = textf2)
    axes4.set_xlim(Xlim1)
    axes4.set_ylim(Ylim1)
    draw_axes(axes4,Xlim1[0],Xlim1[1],Ylim1[0],Ylim1[1])
    axes4.legend()

    global canvas4
    canvas4 = FigureCanvasTkAgg(fig4,self)
    canvas4.show()
    canvas4.get_tk_widget().place(x = 200, y = y2)



    #Axes5 is the plot of the (inverse) Fourier transform of g(t) (G(jw)) calculated by the function FourierTransform
    fig5 = plt.figure(figsize=(3.5,2.5))
    global axes5
    axes5 = fig5.add_subplot(111)
    axes5.clear()

    Xsf,Ysf,Xlim2,Ylim2 = FourierTransform(sf,TF)
    if sf == 3:
        Xsf2 = Xsf[1]
        Ysf2 = Ysf[1]
        axes5.plot(Xsf2,Ysf2,'b')
        Xsf = Xsf[0]
        Ysf = Ysf[0]
        if TF == 1:
            axes5.text(-3,2.5,'$(j\pi)$',fontsize = 8)
            axes5.text(2,-3,'$(-j\pi)$',fontsize = 8)
        elif TF ==2:
            axes5.text(-3,-0.5,"$(-j1/2)$")
            axes5.text(2,0.4,"$(j1/2)$")
    axes5.plot(Xsf,Ysf,'b',label = textg2)
    axes5.set_xlim(Xlim2)
    axes5.set_ylim(Ylim2)
    draw_axes(axes5,Xlim2[0],Xlim2[1],Ylim2[0],Ylim2[1])
    axes5.legend()
    global canvas5
    canvas5 = FigureCanvasTkAgg(fig5,self)
    canvas5.show()
    canvas5.get_tk_widget().place(x = 520, y = y2)



    #Axes6 is the plot of the multiplication calculated by the function Multiplication
    fig6 = plt.figure(figsize=(3.5,2.5))
    global axes6
    axes6 = fig6.add_subplot(111)
    axes6.clear()

    Hx,Hy,Xlim3,Ylim3 = Multiplication(ff,sf,TF,Xff,Xsf,Yff,Ysf)
    if ff == 4 and sf<3:
        axes6.plot(Hx[0],Hy[0],'g')
        axes6.plot(Hx[1],Hy[1],'g')
        axes6.plot(Hx[2],Hy[2],'g')
        axes6.plot(Hx[3],Hy[3],'g')
        axes6.plot(Hx[4],Hy[4],'g')
        axes6.plot(Hx[5],Hy[5],'g')
        axes6.plot(Hx[6],Hy[6],'g',label = texth2)
        axes6.plot(Hx[7],Hy[7],'y--')

    elif (ff == 3 or sf == 3) and ff != 5 and sf != 5:
        Hx2 = Hx[1]
        Hy2 = Hy[1]
        axes6.plot(Hx2,Hy2,'g')
        Hx = Hx[0]
        Hy = Hy[0]
        axes6.plot(Hx,Hy,'g',label = texth2)
        if TF==1:
            if (ff==3 and sf==1) or (ff==1 and sf==3):
                axes6.text(-3,1.8,"$(j2)$")
                axes6.text(2,-2,"$(-j2)$")
            elif (ff==3 and sf==2) or (ff==2 and sf==3):
                axes6.text(-3,4/np.pi-0.2,"$(j4/\pi)$")
                axes6.text(1.5,-4/np.pi,"$(-j4/\pi)$")
            elif ff==3 and sf==3:
                axes6.text(-3,-np.pi**2,"$(-\pi^2\infty)$")
                axes6.text(1.5,-np.pi**2,"$(-\pi^2\infty)$")
        elif TF == 2:
            if (ff==3 and sf==1) or (ff==1 and sf==3):
                axes6.text(-3,-0.9/np.pi,"$(-j1/\pi)$")
                axes6.text(2,1/np.pi-0.02,"$(j1/\pi)$")
            elif (ff==3 and sf==2) or (ff==2 and sf==3):
                axes6.text(-3,-2/np.pi**2,"$(-j2/\pi^2)$")
                axes6.text(1.5,2/np.pi**2-0.02,"$(j2/\pi^2)$")
            elif ff==3 and sf==3:
                axes6.text(-3,-np.pi/2,"$(\pi/2\infty)$")
                axes6.text(1.5,-np.pi/2,"$(\pi/2\infty)$")

    else:
        axes6.plot(Hx,Hy,'g',label = texth2)


    axes6.set_xlim(Xlim3)
    axes6.set_ylim(Ylim3)
    draw_axes(axes6,Xlim3[0],Xlim3[1],Ylim3[0],Ylim3[1])
    axes6.legend()


    global canvas6
    canvas6 = FigureCanvasTkAgg(fig6,self)
    canvas6.show()
    canvas6.get_tk_widget().place(x = 870, y = y2)

    #Axes labels
    ttk.Label(self,text = "t [s]").place(x = 450, y=280)
    ttk.Label(self,text = "t [s]").place(x = 770, y=280)
    ttk.Label(self,text = "t [s]").place(x = 1120, y=280)
    ttk.Label(self,text = "w [rad/s]").place(x = 450, y=570)
    ttk.Label(self,text = "w [rad/s]").place(x = 770, y=570)
    ttk.Label(self,text = "w [rad/s]").place(x = 1120, y=570)


def draw_axes(axes,xlim1,xlim2,ylim1,ylim2):
    """
    Drawing an x-axis
    """

    xdist = xlim2-xlim1
    ydist = ylim2-ylim1

    #Plotting the x-axis
    x1 = [xlim1,xlim2,xlim2-xdist/40.0,xlim2,xlim2-xdist/40.0]
    y1 = [0,0,0-ydist/40.0,0,0+ydist/40.0]
    axes.plot(x1,y1,'k')


def CalcConvolution(ff,sf):
    """
    Calculating the convolution of the two given functions.
    ff is the number of the first function
    sf is the number of the second function
    """


    if ff < 3 and sf < 3:  #rectangle and triangle

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        xcon = np.linspace(-1,1,len(xff)+len(xsf)-1)

    elif ff == 4 and sf == 1:        #Impulse train and rectangle

        xcon = [-2.9,-2.9,-1.9,-1.9,-1.7,-1.7,-0.7,-0.7,-0.5,-0.5,0.5,0.5,0.7,0.7,1.7,1.7,1.9,1.9,2.9,2.9]
        ycon = [0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,0]

    elif ff == 4 and sf == 2:         #Impulse train and triangle

        xcon = [-2.9,-2.4,-1.9,-1.7,-1.2,-0.7,-0.5,0,0.5,0.7,1.2,1.7,1.9,2.4,2.9]
        ycon = [0,1,0,0,1,0,0,1,0,0,1,0,0, 1, 0]

    elif ff == 4 and sf == 3:           #Impulse train and sinus

        xcon = [-2.5,2.5]
        ycon = [0,0]

    elif (ff == 3 and sf < 3) or (ff<3 and sf ==3) :            #Sinus and rectangle/triangle

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        xcon = np.linspace(-5.5,5.5,len(ycon))


    elif ff == 3 and sf == 3:         #Sinus and sinus

        ycon = np.convolve(yff,ysf)*(xsf[1]-xsf[0])
        xcon = np.linspace(-10,10,len(ycon))

    elif (ff == 5 and sf < 3) or (ff < 3 and sf == 5):

        ycon = np.convolve(yff,ysf)*(min(xsf[1]-xsf[0],xff[1]-xff[0]))
        xcon = np.linspace(-6*np.pi-0.5,6*np.pi+0.5,len(ycon))

    elif (ff == 5 and sf == 3) or (ff == 3 and sf == 5):

        xcon = [-6*np.pi,6*np.pi]
        ycon = [0,0]

    elif ff == 4 and sf == 5:

        xcon = [-6*np.pi,6*np.pi]
        ycon = [2*np.pi/1.2,2*np.pi/1.2]

    elif ff == 5 and sf == 5:

        xcon = np.linspace(-6*np.pi,6*np.pi,1000)
        ycon = 2*np.pi*2*np.sin(xcon/2)/xcon

    return xcon,ycon


def FourierTransform(f,TF):
    """
    Calculating the (inverse) Fourier transform of the given function.
    f is the number of the function
    TF is 1 if f is a function of time t, and 2 if f is a function of frequency w
    """
    if TF == 1:
        if f == 1: #rectangle
            Xf = np.linspace(-6*np.pi,6*np.pi,1000)
            Yf = np.divide(2*np.sin(Xf/2),Xf)
            Xlim = [-6*np.pi,6*np.pi]
            Ylim = [-0.5,1.5]
        elif f == 2: #triangle
            Xf = np.linspace(-6*np.pi,6*np.pi,1000)
            Yf = np.multiply(np.divide(2*np.sin(Xf/2),Xf),np.divide(2*np.sin(Xf/2),Xf))
            Xlim = [-6*np.pi,6*np.pi]
            Ylim = [-0.5,1.5]
        elif f == 3: #sinus
            Xf = [[-np.pi-0.1,-np.pi,-np.pi+0.1,-np.pi,-np.pi],[np.pi,np.pi,np.pi-0.1,np.pi,np.pi+0.1]]
            Yf = [[np.pi-0.1,np.pi,np.pi-0.1,np.pi,0],[0,-np.pi,-np.pi+0.1,-np.pi,-np.pi+0.1]]
            Xlim = [-4,4]
            Ylim = [-4,4]
        elif f == 4: #impulse train
            Xf = [-2*np.pi/1.2,]*100 + list(np.linspace(-2*np.pi/1.2-0.2,-2*np.pi/1.2,10)) + list(np.linspace(-2*np.pi/1.2,-2*np.pi/1.2+0.2,10))
            Yf = list(np.linspace(0,2*np.pi/1.2,100))+list(np.linspace(2*np.pi/1.2-0.2,2*np.pi/1.2,10)) + list(np.linspace(2*np.pi/1.2,2*np.pi/1.2-0.2,10))
            Xlim = [-2*np.pi/1.2-1,2*np.pi/1.2+1]
            Ylim = [0,2+2*np.pi/1.2]
        elif f == 5: #Sinc
            Xf = [-2.5,-0.5,-0.5,0.5,0.5,2.5]
            Yf = [0,0,2*np.pi,2*np.pi,0,0]
            Xlim = [-2.5,2.5]
            Ylim = [0, 8]

    if TF == 2:
        if f ==1:   #rectangle
            Xf = np.linspace(-6*np.pi,6*np.pi,1000)
            Yf = np.divide(2*np.sin(Xf/2),Xf)/2/np.pi
            Xlim = [-6*np.pi,6*np.pi]
            Ylim = [-0.5/2/np.pi,1.5/2/np.pi]
        elif f ==2: #triangle
            Xf = np.linspace(-6*np.pi,6*np.pi,1000)
            Yf = np.multiply(np.divide(2*np.sin(Xf/2),Xf),np.divide(2*np.sin(Xf/2),Xf))/2/np.pi
            Xlim = [-6*np.pi,6*np.pi]
            Ylim = [-0.5/2/np.pi,1.5/2/np.pi]
        elif f == 3:  #Sinus
            Xf = [[-np.pi-0.1,-np.pi,-np.pi+0.1,-np.pi,-np.pi],[np.pi,np.pi,np.pi-0.1,np.pi,np.pi+0.1]]
            Yf = [list(-np.asarray([np.pi-0.1,np.pi,np.pi-0.1,np.pi,0])/2/np.pi),list(-np.asarray([0,-np.pi,-np.pi+0.1,-np.pi,-np.pi+0.1])/2/np.pi)]
            Xlim = [-4,4]
            Ylim = [-4/2/np.pi*1.5,4/2/np.pi*1.5]
        elif f == 4:  #Impulse train
            Xf = [-2*np.pi/1.2,]*100 + list(np.linspace(-2*np.pi/1.2-0.2,-2*np.pi/1.2,10)) + list(np.linspace(-2*np.pi/1.2,-2*np.pi/1.2+0.2,10))
            Yf = np.asarray(list(np.linspace(0,2*np.pi/1.2,100))+list(np.linspace(2*np.pi/1.2-0.2,2*np.pi/1.2,10)) + list(np.linspace(2*np.pi/1.2,2*np.pi/1.2-0.2,10)))/2/np.pi
            Xlim = [-2*np.pi/1.2-1,2*np.pi/1.2+1]
            Ylim = np.asarray([0,2+2*np.pi/1.2])/2/np.pi
        elif f == 5:   #Sinc
            Xf = [-2.5,-0.5,-0.5,0.5,0.5,2.5]
            Yf = [0,0,1,1,0,0]
            Xlim = [-2.5,2.5]
            Ylim = [0, 1.5]


    return Xf,Yf,Xlim,Ylim


def Multiplication(ff,sf,TF,Xff,Xsf,Yff,Ysf):
    """
    Function that calculates the multiplication of the two given functions. It also adds a factor 2pi if the convolution
    is calculated in the time domain.
    ff is the number of the first function
    TF is 1 if the convolution happened in the time domain, and 2 if it happened in the frequency domain. The multiplication
    is in the other domain, so frequency if TF == 1 and time if TF == 2.
    """
    if TF == 1:
        A=1
        B=1
    elif TF == 2:
        A = 2*np.pi
        B = -1

    if ff <3 and sf < 3:   #rectangle and triangle

        Hx = Xff
        Hy = np.multiply(Yff,Ysf)*A
        Xlim = [-6*np.pi,6*np.pi]
        Ylim = [-0.5/A,2/A]

    elif (ff == 3 and sf == 1) or (ff==1 and sf ==3):    #sinus and rectangle

        a = 2*np.sin(np.pi/2)/np.pi
        Hx = [list(B*np.asarray([-np.pi-0.1,-np.pi,-np.pi+0.1,-np.pi,-np.pi])),list(B*np.asarray([np.pi,np.pi,np.pi-0.1,np.pi,np.pi+0.1]))]
        Hy = [list(np.asarray([np.pi-0.1,np.pi,np.pi-0.1,np.pi,0])*a/A),list(np.asarray([0,-np.pi,-np.pi+0.1,-np.pi,-np.pi+0.1])*a/A)]
        Xlim = [-4,4]
        Ylim = list(np.asarray([-6,6])*a/A)

    elif (ff == 3 and sf == 2) or (ff==2 and sf ==3):   #sinus and triangle

        a = (2*np.sin(np.pi/2)/np.pi)**2
        Hx = [list(B*np.asarray([-np.pi-0.1,-np.pi,-np.pi+0.1,-np.pi,-np.pi])),list(B*np.asarray([np.pi,np.pi,np.pi-0.1,np.pi,np.pi+0.1]))]
        Hy = [list(np.asarray([np.pi-0.1,np.pi,np.pi-0.1,np.pi,0])*a/A),list(np.asarray([0,-np.pi,-np.pi+0.1,-np.pi,-np.pi+0.1])*a/A)]
        Xlim = [-4,4]
        Ylim = list(np.asarray([-6,6])*a/A)

    elif ff == 3 and sf == 3:   #sinus and sinus

        Hx = [[-np.pi-0.1,-np.pi,-np.pi+0.1,-np.pi,-np.pi],[np.pi,np.pi,np.pi-0.1,np.pi,np.pi+0.1]]
        Hy = [list(-np.asarray([np.pi-0.1,np.pi,np.pi-0.1,np.pi,0])*np.pi/A),list(np.asarray([0,-np.pi,-np.pi+0.1,-np.pi,-np.pi+0.1])/A*np.pi)]
        Xlim = [-4,4]
        Ylim = [-4*np.pi*1.5/A,4*np.pi*1.5/A]

    elif ff == 4 and sf == 1:   #impulse train and rectangle

        a = 2*np.sin(2*np.pi/1.2/2)/2/np.pi*1.2
        Xf = [-2*np.pi/1.2,]*100 + list(np.linspace(-2*np.pi/1.2-0.2,-2*np.pi/1.2,10)) + list(np.linspace(-2*np.pi/1.2,-2*np.pi/1.2+0.2,10))
        Yf = list(np.linspace(0,2*np.pi/1.2,100))+list(np.linspace(2*np.pi/1.2-0.2,2*np.pi/1.2,10)) + list(np.linspace(2*np.pi/1.2,2*np.pi/1.2-0.2,10))
        Xf1 = np.asarray(Xf)
        Yf1 = a*np.asarray(Yf)/2/np.pi*1.2
        Xf2 = Xf1 + 2*np.pi/1.2
        Yf2 = np.asarray(Yf)/2/np.pi*1.2
        Xf3 = Xf2 + 2*np.pi/1.2
        Yf3 = Yf1
        Xf4 = Xf3 + 2*np.pi/1.2
        Yf4 = Yf1 * 8*np.sin(4*np.pi/1.2/2)/4/np.pi*1.2**2
        Xf5 = Xf1 - 2*np.pi/1.2
        Yf5 = Yf4
        Xf6 = Xf4 + 2*np.pi/1.2
        Yf6 = Yf1 * 8*np.sin(6*np.pi/1.2/2)/6/np.pi*1.2**2
        Xf7 = Xf5 - 2*np.pi/1.2
        Yf7 = Yf6

        Hx = [list(Xf1),list(Xf2),list(Xf3),list(Xf4),list(Xf5),list(Xf6),list(Xf7),list(Xsf)]
        Hy = [list(Yf1),list(Yf2),list(Yf3),list(Yf4),list(Yf5),list(Yf6),list(Yf7),list(np.asarray(Ysf)*A)]
        Xlim = [-6*np.pi,6*np.pi]
        Ylim = list(np.asarray([-1,1.5]))

    elif ff == 4 and sf == 2:   #impulse train and triangle

        a = (2*np.sin(2*np.pi/1.2/2)/2/np.pi*1.2)**2
        Xf = [-2*np.pi/1.2,]*100 + list(np.linspace(-2*np.pi/1.2-0.2,-2*np.pi/1.2,10)) + list(np.linspace(-2*np.pi/1.2,-2*np.pi/1.2+0.2,10))
        Yf = list(np.linspace(0,2*np.pi/1.2,100))+list(np.linspace(2*np.pi/1.2-0.2,2*np.pi/1.2,10)) + list(np.linspace(2*np.pi/1.2,2*np.pi/1.2-0.2,10))
        Xf1 = np.asarray(Xf)
        Yf1 = a*np.asarray(Yf)/2/np.pi*1.2
        Xf2 = Xf1 + 2*np.pi/1.2
        Yf2 = np.asarray(Yf)/2/np.pi*1.2
        Xf3 = Xf2 + 2*np.pi/1.2
        Yf3 = Yf1
        Xf4 = Xf3 + 2*np.pi/1.2
        Yf4 = Yf1 * (8*np.sin(4*np.pi/1.2/2)/4/np.pi*1.2**2)**2
        Xf5 = Xf1 - 2*np.pi/1.2
        Yf5 = Yf4
        Xf6 = Xf4 + 2*np.pi/1.2
        Yf6 = Yf1 * (8*np.sin(6*np.pi/1.2/2)/6/np.pi*1.2**2)**2
        Xf7 = Xf5 - 2*np.pi/1.2
        Yf7 = Yf6

        Hx = [list(Xf1),list(Xf2),list(Xf3),list(Xf4),list(Xf5),list(Xf6),list(Xf7),list(Xsf)]
        Hy = [list(Yf1),list(Yf2),list(Yf3),list(Yf4),list(Yf5),list(Yf6),list(Yf7),list(np.asarray(Ysf)*A)]
        Xlim = [-6*np.pi,6*np.pi]
        Ylim = list(np.asarray([-1,1.5]))

    elif ff==4 and sf == 3:     #impulse train and sinus

        Hx = [-2*np.pi,2*np.pi]
        Hy = [0, 0]
        Xlim = [-2*np.pi,2*np.pi]
        Ylim = [-2*np.pi,2*np.pi]

    elif ff == 5 and sf == 1 or ff == 1 and sf == 5:    #sinc and rectangle

        Hx = [-0.5,]+list(np.linspace(-0.5,0.5,1000))+[0.5,]
        Hy = [0,]+ list(2*np.sin(np.linspace(-0.5,0.5,1000)/2)/np.linspace(-0.5,0.5,1000)*2*np.pi/A)+[0,]
        Xlim = [-2.5, 2.5]
        Ylim = [0, 9/A]

    elif ff == 5 and sf == 2 or ff == 2 and sf == 5:    #sinc and triangle

        Hx = [-0.5,]+list(np.linspace(-0.5,0.5,1000))+[0.5,]
        Hy = [0,]+ list((2*np.sin(np.linspace(-0.5,0.5,1000)/2)/np.linspace(-0.5,0.5,1000))**2*2*np.pi/A)+[0,]
        Xlim = [-2.5, 2.5]
        Ylim = [0, 9/A]

    elif ff == 5 and sf == 3 or ff == 3 and sf == 5:    #sinc and sinus

        Hx = [-6*np.pi,6*np.pi]
        Hy = [0,0]
        Xlim = [-6*np.pi,6*np.pi]
        Ylim = [0,2*np.pi/A]

    elif ff == 4 and sf == 5:   #impulse train and sinc

        Hx = [0,0,-((2*np.pi)**2/1.2)/100,0,((2*np.pi)**2/1.2)/100]
        Hy = [0,(2*np.pi)**2/1.2/A,(2*np.pi)**2/1.2/A-(2*np.pi)**2/1.2/A/20,(2*np.pi)**2/1.2/A,(2*np.pi)**2/1.2/A-(2*np.pi)**2/1.2/A/20]
        Xlim = [-2*np.pi,2*np.pi]
        Ylim = [0,(2*np.pi)**2/1.2/A*1.2]

    elif ff == 5 and sf == 5:   #sinc and sinc

        Hx = [-2.5,-0.5,-0.5,0.5,0.5,2.5]
        Hy = [0,0,(2*np.pi)**2/A,(2*np.pi)**2/A,0,0]
        Xlim = [-2.5,2.5]
        Ylim = [0, (2*np.pi)**2/A*1.2]

    return Hx,Hy,Xlim,Ylim






#THIRD PART: THE BUILDING OF THE TKINTER APPLICATION
"""
Here the application and the different pages are defined as classes.
"""
class Application(tk.Tk):

    def __init__(self, *args, **kwargs):
        """
        The main application. Here the geometry can be changed.
        If you want to add a page, you define it as a new class (see below) and don't forget to add it to the list:
        for F in (...)
        """

        tk.Tk.__init__(self,*args,**kwargs)
        tk.Tk.title(self,"Convolution")
        tk.Tk.geometry(self,"1200x650+100+50")

        container = tk.Frame(self)
        container.pack(side="top", fill = "both", expand=True)

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (HomePage, PageOne, PageTwo,PageThree):

            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(HomePage)

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()



class HomePage(tk.Frame):

    def __init__(self, parent, controller):
        """
        Creation of the Home Page with the initial instructions
        """

        tk.Frame.__init__(self,parent)
        label1 = ttk.Label(self, text="HOME PAGE",font=LARGE_FONT)
        label1.place(x=550,y=10)

        label2 = ttk.Label(self, text = "Basic Instructions", font=LARGE_FONT ).place(x=100,y=100)
        T = tk.Text(self,height = 15, width = 110)
        T.place(x=100,y=150)
        T.insert(tk.END,"This is the home page of this application.\nFrom here you can basically go to the two interesting pages.\
\n\nThe first page you can use to get to know the basic idea of a convolution. You get to select the two functions\n\
of which you wish to calculate the convolution, and then you can use the slider next to t to see the magic\nhappen. \
Note that we work with a finite version of the sinus, and of the impulse train that should contain an\
 infinite amount of impulses, we only draw 3.\n\n\
On the second page you can play around with convolutions in the time and the frequency domain. You should\nremember from the course\
 (if you have already covered it) that the convolution in one domain corresponds to a multiplication in the other. This will be \
clear to you once and for all when you have checked page 2. Note    that we consider the sinus here to be infinitely long, so the \
fourier transform gives us two dirac delta's.   The same applies to the impulse train, which is infinitely long, but we only show \
three of them.\n\nThe buttons below will bring you to the respective pages. Have fun!")


        button1 = ttk.Button(self, text="BASICS OF CONVOLUTION",command=lambda: \
                           controller.show_frame(PageOne)).place(x=100,y=420)

        button2 = ttk.Button(self, text="CONVOLUTION IN THE TIME AND FREQUENCY DOMAINS",command=lambda: \
                           controller.show_frame(PageTwo)).place(x=100,y=450)

class PageOne(tk.Frame):

    def __init__(self,parent,controller):
        """
        Creation of the first page.
        """

        tk.Frame.__init__(self,parent)
        label1 = ttk.Label(self, text="BASICS OF CONVOLUTION",font=LARGE_FONT)
        label1.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="BACK TO HOME",command=lambda:\
                           controller.show_frame(HomePage)).place(x = 1070,y=600)

        button2 = ttk.Button(self, text="CONVOLUTION IN THE TIME AND FREQUENCY DOMAINS",command=lambda: \
                           controller.show_frame(PageTwo)).place(x = 850,y=625)


        #Defining the Radiobuttons
        MODES = [
        ("Rectangle", 1),
        ("Triangle", 2),
        ("Sinus", 3),
        ("Impulse train", 4),
        ]

        f1 = tk.IntVar()
        f1.set(1) # initialize

        i = 0
        for text, mode in MODES:
            b = tk.Radiobutton(self, text=text,
                            variable=f1, value=mode)
            b.place(x=10,y=150+i*20)
            i += 1

        MODES = [
        ("Rectangle", 1),
        ("Triangle", 2),
        ("Sinus", 3),
        ]

        f2 = tk.IntVar()
        f2.set(1) # initialize

        i = 0
        for text, mode in MODES:
            b = tk.Radiobutton(self, text=text,
                            variable=f2, value=mode)
            b.place(x=120,y=150+i*20)
            i += 1



        label2 = ttk.Label(self, text = "f(t)",font=('Verdana',10)).place(x = 40,y = 120)
        label3 = ttk.Label(self, text = "g(t)",font=('Verdana',10)).place(x = 130,y = 120)
        label4 = ttk.Label(self, text = "First choose the two functions f(t) and g(t) from the options below. Then a slider for the time t will appear, and the convolution h(t) = (f*g)(t) = int[f(tau).g(t-tau)dtau] will be calculated and plotted in green for every value of t.").place(x=10,y=40)
        button3 = tk.Button(self, text="OK",command=lambda : ConVol(self,f1.get(),f2.get())).place(x=10, y=260)
        button4 = tk.Button(self, text="Plot the resulting convolution", command=lambda: PlotConVol(self,f1.get(),f2.get())).place(x=10,y=300)

        T = tk.Text(self, height = 30, width = 20)
        T.place(x = 1020, y = 100)
        T.insert(tk.END,"Note that:          -the rectangle and  triangle are        (contrary to the    next tab and the    course notes) not   symmetric around they-axis, but defined from 0 to 1!        -The sinus does not go from -inf to inf,but instead is a    finite version. Thisalso explains the   sudden drop in the  convolution of the  impulse train and   the sinus.")


class PageTwo(tk.Frame):

    def __init__(self,parent,controller):
        """
        Creating the second page
        """

        tk.Frame.__init__(self,parent)
        label = ttk.Label(self, text="CONVOLUTION IN THE TIME AND FREQUENCY DOMAINS",font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="BACK TO HOME",command=lambda:\
                           controller.show_frame(HomePage)).place(x = 1000,y=590)


        button2 = ttk.Button(self, text="BASICS OF CONVOLUTION",command=lambda:\
                           controller.show_frame(PageOne)).place(x = 1000,y=620)

        button3 = ttk.Button(self, text ="Explanation for sinus and sinus", command = lambda:\
                           controller.show_frame(PageThree)).place(x = 10, y = 570)



        TF = tk.IntVar()

        b1 = tk.Radiobutton(self, text="Time", variable= TF, value=1, command= lambda: PrintTime(self) )
        b1.place(x = 50, y = 80)
        b2 = tk.Radiobutton(self, text="Frequency", variable= TF, value=2, command = lambda: PrintFreq(self) )
        b2.place(x = 50, y = 100)
        label2 = ttk.Label(self, text = "Convolution in time or frequency domain?").place(x=10,y=60)
        TF.set(1)

        MODES = [
        ("Rectangle", 1),
        ("Triangle", 2),
        ("Sinus", 3),
        ("Impulse train", 4),
        ("Sinc",5)
        ]

        f1 = tk.IntVar()
        f1.set(1) # initialize

        i = 0
        for text, mode in MODES:
            b = tk.Radiobutton(self, text=text,
                            variable=f1, value=mode)
            b.place(x=10,y=250+i*20)
            i += 1

        MODES = [
        ("Rectangle", 1),
        ("Triangle", 2),
        ("Sinus", 3),
        ("Sinc",5)
        ]

        f2 = tk.IntVar()
        f2.set(1) # initialize

        i = 0
        for text, mode in MODES:
            b = tk.Radiobutton(self, text=text,
                            variable=f2, value=mode)
            b.place(x=120,y=250+i*20)
            i += 1



        button5 = tk.Button(self, text="OK",command=lambda : ConVolTF(self,f1.get(),f2.get(),TF.get())).place(x=10, y=360)


class PageThree(tk.Frame):

    def __init__(self,parent,controller):
        """
        Creating the third page (Explanation of the convolution of a sinus and a sinus)
        """
        tk.Frame.__init__(self,parent)
        label = ttk.Label(self, text="Explanation for sinus and sinus",font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        T = tk.Text(self, height = 25, width = 130)
        T.place(x = 100, y = 100)
        T.insert(tk.END,'We can actually calculate the convolution integral: int(sin(tau)sin(t-tau)dtau,-inf,inf) in this case, using some basic           goniometrical formulas: \ncos(a+b) = cos(a)cos(b) - sin(a)sin(b)       and      cos(a-b) = cos(a)cos(b) - sin(a)sin(b). This yields:\n\nsin(a)sin(b) = (cos(a-b) - cos(a+b)) / 2\n\nThus int(sin(tau)sin(t-tau) dtau) = 1/2*int([cos(2tau-t) - cos(t)] dtau)\n				  = 1/2* [sin(2tau-t)/2] - 1/2* [tau*cos(t)]\n\nSubstituting tau = inf and tau = -inf, we see that the first term becomes negligible to the second one, which goes to infinity. Sothe final result is a -cos(t) with an amplitude of infinity. In our case the graph is finite because the sine signal we use to    calculate the convolution is limited in time, however it will go to infinity for a regular sine signal.\n\nWe can also see this in the frequency domain. Here we multiply two dirac pulses that overlap. You probably know that a            characteristic of a dirac pulse is the fact that if you multiply it with a function, the result is a new dirac pulse (that goes toinfinity), where the amplitude is multiplied by the value of the function at the time of the impulse. This is in fact what we callsampling.\n\nHere the value of the "function", which is the other dirac impulse, is j*pi*inf (or simply inf if you want). So the result is two new dirac pulses with amplitude infinity. This explains why we have a cosinus with an amplitude of infinity. ')

        button2 = ttk.Button(self, text="BACK",command=lambda: \
                           controller.show_frame(PageTwo)).place(x = 1000,y=600)



#Making everything run
app = Application()
app.mainloop()